# Pré-processamento

In [16]:
import numpy as np
import pandas as pd

doencas = pd.read_csv("dados/doencas.csv")
bg = pd.read_csv("dados/background.csv")

# removendo atributos sensíveis
doencas.drop(['id', 'doenca'], axis=1, inplace=True)

# inserindo coluna para generalizar localização
doencas.insert(0, 'localizacao', '*')

# doencas.head()
# bg.head()

# Funções de anonimização

## Anonimizando gênero

In [12]:
def anon_genero(data):
    for i in data.index:
        if(data.at[i, 'genero'] == 'Female'):
            data.at[i, 'genero'] = 0
        else:
            data.at[i, 'genero'] = 1

anon_genero(doencas)
doencas.head()

,genero,data,cidade,estado
0,0,31/01/2000,Brooklyn,New York
1,0,16/10/1999,Topeka,Kansas
2,0,10/10/1999,Schenectady,New York
3,0,09/01/2000,New York City,New York
4,1,03/11/1999,Syracuse,New York


## Anonimizando localização

In [17]:
def anon_local(data):
    for i in data.index:
        cid = data.at[i, 'cidade']
        est = data.at[i, 'estado']
        
        data.at[i, 'localizacao'] = [cid, est]
    data.drop(['cidade', 'estado'], axis=1, inplace=True)
        
anon_local(doencas)
doencas.head()

,localizacao,genero,data
0,"[Brooklyn, New York]",Female,31/01/2000
1,"[Topeka, Kansas]",Female,16/10/1999
2,"[Schenectady, New York]",Female,10/10/1999
3,"[New York City, New York]",Female,09/01/2000
4,"[Syracuse, New York]",Male,03/11/1999


## Separando data

In [ ]:
def anon_date(data):
    for()